<a href="https://colab.research.google.com/github/newfull5/AI-Project/blob/master/CNN_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.0002
num_epoch = 10

In [2]:
mnist_train = dset.MNIST("./", train=True, transform = transforms.ToTensor(),target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform = transforms.ToTensor(),target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
class CNN(nn.Module):

  def __init__(self):
    super(CNN,self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1,16,5),
        nn.ReLU(),
        nn.Conv2d(16,32,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(32,64,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3,100),
        nn.ReLU(),
        nn.Linear(100,10)
    )

  def forward(self,x):
    out = self.layer(x)
    out = out.view(batch_size, -1)
    out = self.fc_layer(out)
    return out

In [0]:
# nn.Conv2D(in_channels, out_chanels, kernel_size)
# [batch_size = 1, in_channels = 1, 가로 = 28, 세로 = 28]  =>  [1, outchannels = 16, 24, 24]

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
loss_arr = []
for i in range(num_epoch):
  for j,[image,label] in enumerate(train_loader):
    x = image.to(device)
    y_= label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()

    if j % 1000 == 0:
      print(loss)
      loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3110, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2466, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2027, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0897, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0664, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0471, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0732, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0632, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0239, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0265, device='cuda:0', grad_fn=<NllLossBackward>)


In [7]:
correct = 0
total = 0

with torch.no_grad():
  for image,label in test_loader:
    x = image.to(device)
    y_= label.to(device)

    output = model.forward(x)
    _,output_index = torch.max(output,1)

    total += label.size(0)
    correct += (output_index == y_).sum().float()

  print('Accuracy of Test Data: {}'.format(100*correct/total))

Accuracy of Test Data: 98.9249496459961
